In [1]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp


Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 43.8MB/s]


In [2]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
df=pd.read_csv('preprocessed_data.csv')#reading the data into DataFrame

In [4]:
df.head(4)#displaying top 4 four data


,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [5]:
def preprocess(x):#removing the last character
  x=x[:-1]
  return x

In [6]:
df['source']=df['source'].apply(preprocess)#preprocessing on source data
df['target']=df['target'].apply(preprocess)#perprocessing on target data

In [7]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [8]:
df.shape#shape of the data


(2000, 2)

In [9]:
df=df[df['source'].apply(len)<170]#removing source datapoints having length greater than equal to 170
df=df[df['target'].apply(len)<200]#removing target datapoints having length greater than equal to 200

In [10]:
df.shape#shape of the data


(1990, 2)

In [11]:
from sklearn.model_selection import train_test_split
X=df['source']
y=df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.01)#splitting the data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1970,)
(20,)
(1970,)
(20,)


In [57]:
X_train.to_csv('X_train.csv')
y_train.to_csv('y_train.csv')
X_test.to_csv('X_test.csv')
y_test.to_csv('y_test.csv')


Target:

In [13]:
target_tokenizer= Tokenizer()#tokenization on target
target_tokenizer.fit_on_texts(y_train)#fitting on ytrain
target_vocab_size= len(target_tokenizer.word_index) + 1#target vocab size
print(len(target_tokenizer.word_index))

3032


In [14]:
target_encoded_docs_train = target_tokenizer.texts_to_sequences(y_train)#converting text to integers
target_encoded_docs_test = target_tokenizer.texts_to_sequences(y_test)#converting text to integers

In [15]:
target_padded_docs_train = pad_sequences(target_encoded_docs_train,padding='post')#padding to maxlength

In [16]:
target_padded_docs_train.shape


(1970, 43)

In [17]:
target_padded_docs_test = pad_sequences(target_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [18]:
target_padded_docs_test.shape


(20, 43)

Source:

In [19]:
source_tokenizer= Tokenizer()#tokenization on source
source_tokenizer.fit_on_texts(X_train)#fitting to X_train
source_vocab_size= len(source_tokenizer.word_index) + 1#source vocab size
print(len(source_tokenizer.word_index))

3703


In [20]:
source_encoded_docs_train = source_tokenizer.texts_to_sequences(X_train)#converting text to sequence
source_encoded_docs_test = source_tokenizer.texts_to_sequences(X_test)#converting text to sequence

In [21]:
source_padded_docs_train = pad_sequences(source_encoded_docs_train,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [22]:
source_padded_docs_train.shape

(1970, 43)

In [23]:
source_padded_docs_test = pad_sequences(source_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [24]:
source_padded_docs_test.shape

(20, 43)

In [25]:
#we are reshaping the dataset because the sparese_categorical_crossentropy requires data to be three dimensional

target_padded_docs_train=target_padded_docs_train.reshape((*target_padded_docs_train.shape,1))
target_padded_docs_test=target_padded_docs_test.reshape((*target_padded_docs_test.shape,1))

In [26]:
print(target_padded_docs_train.shape)
print(target_padded_docs_test.shape)

(1970, 43, 1)
(20, 43, 1)


In [27]:
#we are reshaping the dataset because the sparese_categorical_crossentropy requires data to be three dimensional

source_padded_docs_train=source_padded_docs_train.reshape((*source_padded_docs_train.shape,1))
source_padded_docs_test=source_padded_docs_test.reshape((*source_padded_docs_test.shape,1))

In [28]:
print(source_padded_docs_train.shape)
print(source_padded_docs_test.shape)

(1970, 43, 1)
(20, 43, 1)


In [59]:
import pandas as pd 
pd.DataFrame(source_encoded_docs_train).to_csv("source_encoded_docs_train.csv")
pd.DataFrame(source_encoded_docs_test).to_csv("source_encoded_docs_test.csv")
pd.DataFrame(target_encoded_docs_train).to_csv("target_encoded_docs_train.csv")
pd.DataFrame(target_encoded_docs_test).to_csv("target_encoded_docs_test.csv")


Model1:

In [32]:
input=tf.keras.layers.Input(shape=(43,))
embed=tf.keras.layers.Embedding(source_vocab_size,512, input_length=source_padded_docs_train.shape[1])(input)
lstm1=tf.keras.layers.LSTM(128, return_sequences=True)(embed)  
dense=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(512, activation='relu'))(lstm1)
drop=tf.keras.layers.Dropout(0.5)(dense)
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(drop)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()                                                                                                                               

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 43)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 43, 512)           1896448   
_________________________________________________________________
lstm_1 (LSTM)                (None, 43, 128)           328192    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 43, 512)           66048     
_________________________________________________________________
dropout_1 (Dropout)          (None, 43, 512)           0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 43, 3033)          1555929   
Total params: 3,846,617
Trainable params: 3,846,617
Non-trainable params: 0
_________________________________________________

In [33]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [34]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=50,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/50
2/2 [==============================] - 2s 682ms/step - loss: 7.6636 - accuracy: 0.3253 - val_loss: 2.8829 - val_accuracy: 0.6814
Epoch 2/50
2/2 [==============================] - 1s 296ms/step - loss: 3.3623 - accuracy: 0.6737 - val_loss: 3.7915 - val_accuracy: 0.6814
Epoch 3/50
2/2 [==============================] - 1s 298ms/step - loss: 3.5440 - accuracy: 0.6738 - val_loss: 3.1268 - val_accuracy: 0.6849
Epoch 4/50
2/2 [==============================] - 1s 295ms/step - loss: 2.9334 - accuracy: 0.6277 - val_loss: 2.1335 - val_accuracy: 0.6907
Epoch 5/50
2/2 [==============================] - 1s 297ms/step - loss: 2.2468 - accuracy: 0.6814 - val_loss: 2.0976 - val_accuracy: 0.6826
Epoch 6/50
2/2 [==============================] - 1s 299ms/step - loss: 2.2337 - accuracy: 0.6770 - val_loss: 2.0702 - val_accuracy: 0.6826
Epoch 7/50
2/2 [==============================] - 1s 295ms/step - loss: 2.1996 - accuracy: 0.6778 - val_loss: 2.0878 - val_accuracy: 0.6895
Epoch 8/50
2/2 [====

In [35]:
x=model.predict(source_padded_docs_test[:1])[0]


In [36]:
#https://machinelearningmastery.com
index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
index_to_words[0] = '<PAD>'

' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])


'hey i am still having stuff <PAD> if you reach <PAD> <PAD> <PAD> help it and me you to <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

In [37]:
print(y_test[:1])


1866    I am still having breakfast. If you reach ther...
Name: target, dtype: object


In [38]:
X_test[:1]


1866    Hey i am still having breakfast eh. If you rea...
Name: source, dtype: object

In [41]:
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])
  return y
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output: ")
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  print(' '.join(y_lst))
  print('>'*180)

Input text: 
Hey i am still having breakfast eh. If you reach there first can help rebecca and me chope seats?
Actual Output: 
I am still having breakfast. If you reach there first can you help me and Rebecca reserve seats?
Predicted Output: 
hey i am still having stuff if you reach help it and me you to
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Huh ü take then how i take bus later... Inside got money a not...
Actual Output: 
If you take then how I take bus later? Inside got money or not?
Predicted Output: 
huh you take then how i take bus me have your a not
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Hi neva worry bout da truth coz the truth will lead me 2 ur heart. It's the least a uniqu

In [44]:
import nltk.translate.bleu_score as bleu

bleu_score=[]
for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  bleu_score.append(bleu.sentence_bleu([b[0].split(),],y_lst))
print(bleu_score)
print("The Average Bleu Score is: ",sum(bleu_score)/20)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[0.2581911684267368, 0.21294807603873017, 0.14463972129203845, 0.7598356856515925, 0.4671379777282001, 0.5623413251903491, 0.30207246566144663, 0.23462350320528, 0.7598356856515925, 0.15909672318073625, 0.17122548504687662, 0.46199933699457096, 0.6389431042462724, 0.43012508513132625, 0.1828175732238544, 0.3081980909598119, 0.36177396082048563, 0.24573784957585945, 0.5502659908318907, 0]
The Average Bleu Score is:  0.3605904404428826


Model2:

In [49]:
input=tf.keras.layers.Input(shape=(43,))
embed=tf.keras.layers.Embedding(source_vocab_size,512, input_length=source_padded_docs_train.shape[1])(input)
lstm1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True))(embed)  
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(lstm1)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 43)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 43, 512)           1896448   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 43, 200)           490400    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 43, 3033)          609633    
Total params: 2,996,481
Trainable params: 2,996,481
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [51]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=50,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/50
2/2 [==============================] - 4s 865ms/step - loss: 7.6696 - accuracy: 0.3222 - val_loss: 5.3397 - val_accuracy: 0.6814
Epoch 2/50
2/2 [==============================] - 1s 279ms/step - loss: 4.5154 - accuracy: 0.6737 - val_loss: 2.4947 - val_accuracy: 0.6814
Epoch 3/50
2/2 [==============================] - 1s 283ms/step - loss: 2.7089 - accuracy: 0.6737 - val_loss: 2.7940 - val_accuracy: 0.6814
Epoch 4/50
2/2 [==============================] - 1s 289ms/step - loss: 2.9144 - accuracy: 0.6737 - val_loss: 2.5952 - val_accuracy: 0.6814
Epoch 5/50
2/2 [==============================] - 1s 282ms/step - loss: 2.7359 - accuracy: 0.6742 - val_loss: 2.3767 - val_accuracy: 0.6802
Epoch 6/50
2/2 [==============================] - 1s 284ms/step - loss: 2.4859 - accuracy: 0.6713 - val_loss: 2.1571 - val_accuracy: 0.6767
Epoch 7/50
2/2 [==============================] - 1s 284ms/step - loss: 2.2431 - accuracy: 0.6746 - val_loss: 2.0513 - val_accuracy: 0.6814
Epoch 8/50
2/2 [====

In [52]:
x=model.predict(source_padded_docs_test[7:8])[0]

In [53]:
index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
index_to_words[0] = '<PAD>'

' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])

"how i know last time this one is on father's what <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

In [54]:
print(y_test[7:8])

128    How I know. Last time this one is on offer.
Name: target, dtype: object


In [55]:
X_test[7:8]

128    How i noe... Last time tis one is on offer wat...
Name: source, dtype: object

In [60]:
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])
  return y
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output: ")
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  print(' '.join(y_lst))
  print('>'*180)

Input text: 
Hey i am still having breakfast eh. If you reach there first can help rebecca and me chope seats?
Actual Output: 
I am still having breakfast. If you reach there first can you help me and Rebecca reserve seats?
Predicted Output: 
hey i am still having work ask if you reach there first can help nap and me to and
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Huh ü take then how i take bus later... Inside got money a not...
Actual Output: 
If you take then how I take bus later? Inside got money or not?
Predicted Output: 
huh you take then how i to bus later you got money a or
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Hi neva worry bout da truth coz the truth will lead me 2 ur heart

In [61]:
import nltk.translate.bleu_score as bleu

bleu_score=[]
for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  bleu_score.append(bleu.sentence_bleu([b[0].split(),],y_lst))
print(bleu_score)
print("The Average Bleu Score is: ",sum(bleu_score)/20)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[0.27447938256311044, 0.22718709780542318, 0.3312232008397192, 0.7598356856515925, 0.4671379777282001, 0.5623413251903491, 0.30228791143745415, 0.3508439695638686, 0.7598356856515925, 0.16527975033438158, 0.17795291340072017, 0.5494128986804837, 0.6147881529512643, 0.4111336169005197, 0.22265046674893665, 0.3050975216056289, 0.6537993517025207, 0.3201518925576873, 0.37991784282579627, 0]
The Average Bleu Score is:  0.39176783220696243
